In [147]:
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim

from data_processing.data_generate import generate_random_matrix

### 1.FM模型预测公式

FM的预测公式包含线性部分和二阶交互部分：

$$
\hat{y}(x) = w_0 + \sum_{i=1}^{n} w_i x_i + \sum_{i=1}^{n} \sum_{j=i+1}^{n} (v_i \cdot v_j) x_i x_j
$$

其中：
- $ w_0 $：全局偏置项
- $ w_i $：第 $ i $ 个特征的线性权重
- $ v_i $：第 $ i $ 个特征的隐向量
- $ x_i $：第 $ i $ 个特征的输入值
- $ v_i \cdot v_j $：第 $ i $ 和第 $ j $ 特征的隐向量内积




**对于上部分进行优化 由$O(kn^2) 到 O(kn)$**
其实观察可以发现,原本隐向量部分只有是部分上三角,可以转化成矩阵的全部元素之和一半再减去矩阵的对角线元臺之和的一半：

$$
\begin{aligned}
\sum_{i=1}^{n} \sum_{j=i+1}^{n} (v_i \cdot v_j) x_i x_j &= \frac{1}{2} \sum_{i=1}^{n} \sum_{j=1}^{n} (v_i \cdot v_j) x_i x_j - \frac{1}{2} \sum_{i=1}^{n} (v_i^2) x_i^2\\
&= \frac{1}{2} \sum_{i=1}^{n} \sum_{j=1}^{n}\sum_{i=1}^{k} (v_{i,f} \cdot v_{j,f}) x_i x_j - \frac{1}{2} \sum_{i=1}^{n} \sum_{i=1}^{k}(v_{i,f}^2) x_i^2\\
&= \frac{1}{2} \sum_{i=1}^{k} \left( \sum_{i=1}^{n} \sum_{j=1}^{n} (v_{if} \cdot v_{jf}) x_i x_j - \sum_{i=1}^{n} (v_{if})^2 x_i^2 \right)\\
&= \frac{1}{2} \sum_{i=1}^{k} \left( \left( \sum_{i=1}^{n} v_{if} x_i \right) \left( \sum_{j=1}^{n} v_{jf} x_j \right) - \sum_{i=1}^{n} (v_{if})^2 x_i^2 \right)\\

&= \frac{1}{2} \sum_{i=1}^{k} \left( \left( \sum_{i=1}^{n} v_{if} x_i \right)^2  - \sum_{i=1}^{n} (v_{if})^2 x_i^2 \right)
\end{aligned}
$$


**优化后预测函数**
$$
\hat{y}(x) = w_0 + \sum_{i=1}^{n} w_i x_i + \frac{1}{2} \sum_{i=1}^{k} ( \sum_{i=1}^{n} v_{if} x_i )^2  - \sum_{i=1}^{n} (v_{if})^2 x_i^2)
$$


### 损失函数

FM模型的损失函数通常是均方误差（MSE）损失函数，表示为：

$$
L(\theta) = \frac{1}{2} \sum_{t=1}^{T} (y_t - \hat{y_t})^2
$$

其中：
- $T$ 是训练样本的数量
- $y_t$ 是第 $t$ 个样本的真实值
- $\hat{y_t}$ 是第 $t$ 个样本的预测值
- $\theta$ 是模型的所有参数（包括线性权重和隐向量）

### 2 FM模型的梯度公式

对于FM模型的参数（包括线性权重 $w$ 和隐向量 $v$），我们需要计算损失函数的梯度，以便使用梯度下降法进行优化。我们可以根据损失函数来推导出每个参数的梯度。

#### 偏置项 $w_0$ 梯度：

$$
\frac{\partial L}{\partial w_0} = \hat{y_t} - y_t
$$

#### 线性权重 $w_i$ 梯度：

$$
\frac{\partial L}{\partial w_i} = (\hat{y_t} - y_t) \cdot x_i
$$

#### 隐向量 $v_i$ 梯度：

隐向量的梯度较为复杂，推导过程如下：

$$
\begin{aligned}
\frac{\partial L}{\partial v_i} = (\hat{y_t} - y_t) \cdot \left( \sum_{j=1}^{n} v_j x_j - v_i x_i \right) \cdot x_i\\
= (\hat{y_t} - y_t) \cdot \left( x_i \sum_{j=1}^{n} v_j x_j - v_i x_i^2\right)  \\
\end{aligned}
$$

In [140]:
class FactorizationMachine:
    def __init__(self, n_features, k, learning_rate=0.01, n_iter=1000):
        """
        初始化因子分解机模型。
        参数：
        - n_features: 特征数量
        - k: 隐向量的维度
        - learning_rate: 学习率
        - n_iter: 迭代次数
        """

        self.n_features = n_features
        self.k = k
        self.learning_rate = learning_rate
        self.n_iter = n_iter

        # 初始化参数
        self.w0 = 0
        self.w = np.zeros(n_features) # 线性权重
        self.V = np.random.normal(scale = 1.0 / k, size = (n_features, k))# 隐向量矩阵


    def _predict_instance(self, x):
        """
        预测单个样本的输出。
        参数：
        - x: 输入特征向量
        返回：
        - 预测值
        """
        linear_terms = self.w0 + np.dot(self.w, x)
        interactions = 0.5 * np.sum(
            np.square(np.dot(x, self.V)) - np.dot(x ** 2, self.V ** 2)
        )
        return linear_terms + interactions
         


    def fit(self,X,y):
        """
        训练因子分解机模型。
        参数：
        - X: 特征矩阵，形状为(n_samples, n_features)
        - y: 目标向量，形状为(n_samples,)
        """

        pbar = tqdm(range(self.n_iter), desc = "Training FM", ncols = 100, unit = "iter")
        for _ in pbar:
            rows, cols = X.shape
            mse = 0
            for i in range(rows):
                prediction = self._predict_instance(X[i])
                yi = y[i]
                error = (prediction - yi).astype(np.float32)
                mse += error ** 2
                

                #更新参数
                self.w0 -= self.learning_rate * error
                self.w -= self.learning_rate * error *X[i]

                # 计算 V 的更新，使用矩阵运算来代替逐元素的运算
                x_square = X[i] ** 2  # (n_features,)
                x_V = np.dot(X[i], self.V)  # (1,k)
                grad_V = error * (
                    #这纬度一致其实是矩阵乘法
                    np.dot(X[i].reshape(-1, 1), x_V.reshape(1, -1)) #(n_features,k)
                    - self.V * x_square.reshape(-1, 1) #(n_features,k) 广播机制
                )
                self.V -= self.learning_rate * grad_V
            pbar.set_postfix({"MSE": mse})

            
    def predict(self,X):
        """
        预测样本的输出。
        参数：
        - X: 特征矩阵，形状为(n_samples, n_features)
        返回：
        - 预测值数组，形状为(n_samples,)
        """
        return np.array([self._predict_instance(xi) for xi in X])
        

    

## 实验模拟
注意由于没有合适的数据集，所以这里我们使用随机生成的数据来模拟实验,并且因为生成的数据不一定存在规律,所以只用生成的数据进行预测,而没有使用测试集来评估模型的性能。

In [141]:
#构建矩阵
rows,cols = 100,8
matrix = generate_random_matrix(rows = rows,cols = cols,min_value = 0, max_value = 5,seed = 42)
np.random.seed(42)
y = np.random.rand(rows,1)
res = pd.DataFrame(np.concatenate((matrix,y),axis = 1),index = range(1,rows + 1),columns = list(range(1,cols + 1)) + ['y'])

In [144]:
FM = FactorizationMachine(n_features = cols,k = 5, learning_rate = 0.001,n_iter = 1000)

In [145]:
FM.fit(matrix,y)

Training FM: 100%|███████████████████████████| 1000/1000 [00:03<00:00, 280.45iter/s, MSE=[7.250213]]


In [146]:
res['y_pred'] = FM.predict(matrix)
res

,1,2,3,4,5,6,7,8,y,y_pred
1,3.0,4.0,2.0,4.0,4.0,1.0,2.0,2.0,0.374540,0.605124
2,2.0,4.0,3.0,2.0,5.0,4.0,1.0,3.0,0.950714,0.527338
3,5.0,5.0,1.0,3.0,4.0,0.0,3.0,1.0,0.731994,0.815019
4,5.0,4.0,3.0,0.0,0.0,2.0,2.0,1.0,0.598658,0.584716
5,3.0,3.0,5.0,5.0,5.0,2.0,3.0,3.0,0.156019,0.523286
...,...,...,...,...,...,...,...,...,...,...
96,5.0,1.0,2.0,4.0,0.0,1.0,1.0,5.0,0.493796,0.555199
97,1.0,2.0,4.0,4.0,0.0,5.0,0.0,1.0,0.522733,0.420619
98,0.0,2.0,4.0,1.0,0.0,5.0,2.0,2.0,0.427541,0.478358
99,0.0,4.0,0.0,1.0,0.0,2.0,0.0,4.0,0.025419,0.125237



### pytroch实现

In [161]:
class FM(torch.nn.Module):
    def __init__(self,n_features, k):
        """
        初始化因子分解机模型。
        参数：
        - n_features: 特征数量
        - k: 隐向量的维度
        """

        super(FM,self).__init__()
        self.n_features = n_features
        self.k = k

        #初始化模型参数
        self.w0 = nn.Parameter(torch.zeros(1))
        self.w = nn.Parameter(torch.zeros(n_features))
        self.V = nn.Parameter(torch.zeros(n_features,k) * 0.01)

    def forward(self, X):
        """
        定义前向传播。
        参数：
        - X: 输入特征矩阵，形状为 (batch_size, n_features)
        返回：
        - 预测值 (batch_size,)
        """

        #线性部分
        linear_terms = self.w0 + torch.matmul(X, self.w)

        # 二阶交互部分 (优化计算)
        interactions = 0.5 * torch.sum(
            torch.pow(torch.matmul(X,self.V),2) - - torch.matmul(torch.pow(X, 2), torch.pow(self.V, 2)), dim=1 
        )

        return linear_terms + interactions


def train_fm(model,optimizer, criterion, X, y, n_epochs):
    """
    训练FM模型。
    参数：
    - model: FM模型实例
    - optimizer: 优化器
    - criterion: 损失函数
    - X: 训练数据 (torch.Tensor)
    - y: 目标值 (torch.Tensor)
    - n_epochs: 训练轮数
    """

    model.train()
    pbar = tqdm(range(n_epochs), desc="Training FM", ncols=100, unit="epoch")
    
    for epoch in pbar:
        predictions = model(X)
        loss = criterion(predictions, y)

        #反向传播   
        optimizer.zero_grad() #清除梯度
        loss.backward()       #计算梯度
        optimizer.step()      #更新参数
        
        #显示损失
        pbar.set_postfix({'Loss':loss.item()})

def predict_fm(model, X):
    model.eval()
    with torch.no_grad():
        predictions = model(X)
    return predictions

In [168]:
#数据准备
n_features = 10
k = 5

torch.manual_seed(42)

X = torch.randn(100, n_features)
y = torch.randn(100)

fm_model = FM(n_features = n_features, k = k)
criterion = nn.MSELoss() # 使用均方误差损失函数
optimizer = optim.Adam(fm_model.parameters(), lr = 0.01)

train_fm(fm_model, optimizer = optimizer, criterion = criterion, X=X, y=y, n_epochs= 100)

Training FM: 100%|████████████████████████████████| 100/100 [00:00<00:00, 643.01epoch/s, Loss=0.984]


In [169]:
y_pred = predict_fm(fm_model,X)

res_torch = pd.DataFrame(X)
res_torch['y'] = y
res_torch['y_pred'] = y_pred

res_torch

,0,1,2,3,4,5,6,7,8,9,y,y_pred
0,tensor(1.9269),tensor(1.4873),tensor(0.9007),tensor(-2.1055),tensor(0.6784),tensor(-1.2345),tensor(-0.0431),tensor(-1.6047),tensor(-0.7521),tensor(1.6487),0.101067,0.237655
1,tensor(-0.3925),tensor(-1.4036),tensor(-0.7279),tensor(-0.5594),tensor(-0.7688),tensor(0.7624),tensor(1.6423),tensor(-0.1596),tensor(-0.4974),tensor(0.4396),-1.309492,-0.170638
2,tensor(-0.7581),tensor(1.0783),tensor(0.8008),tensor(1.6806),tensor(1.2791),tensor(1.2964),tensor(0.6105),tensor(1.3347),tensor(-0.2316),tensor(0.0418),-0.410358,0.360829
3,tensor(-0.2516),tensor(0.8599),tensor(-1.3847),tensor(-0.8712),tensor(-0.2234),tensor(1.7174),tensor(0.3189),tensor(-0.4245),tensor(0.3057),tensor(-0.7746),0.468094,-0.080337
4,tensor(-1.5576),tensor(0.9956),tensor(-0.8798),tensor(-0.6011),tensor(-1.2742),tensor(2.1228),tensor(-1.2347),tensor(-0.4879),tensor(-0.9138),tensor(-0.6581),-0.234628,-0.057512
...,...,...,...,...,...,...,...,...,...,...,...,...
95,tensor(0.5692),tensor(-0.7911),tensor(-0.1990),tensor(-1.3616),tensor(-0.5194),tensor(0.0765),tensor(0.3401),tensor(1.4557),tensor(-0.3461),tensor(-0.2634),-0.014565,-0.286557
96,tensor(-0.4477),tensor(-0.7288),tensor(-0.1607),tensor(-0.3206),tensor(-0.6308),tensor(-0.7888),tensor(1.3062),tensor(-0.9276),tensor(-0.2627),tensor(0.9315),-0.748688,-0.001467
97,tensor(-0.4593),tensor(-0.9419),tensor(-0.7089),tensor(2.1861),tensor(-0.6493),tensor(0.4521),tensor(0.8521),tensor(-1.6947),tensor(1.1806),tensor(-2.8929),-0.384403,-0.337158
98,tensor(-0.3876),tensor(-0.7124),tensor(-1.6171),tensor(-0.3590),tensor(-0.4137),tensor(-0.5285),tensor(-0.5082),tensor(1.1478),tensor(0.2401),tensor(-0.7907),0.143502,-0.181565
